In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [2]:
import json
import os
import mp
from tqdm import tqdm

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [3]:
with open('prepare-force-alignment.json') as fopen:
    d = json.load(fopen)
len(d)

510940

In [5]:
r = d[0]
f = r['audio_filename']
new_f = f.replace('/', '_').replace('.mp3', '.json').replace('.wav', '.json')
filename = os.path.join('force_alignment', new_f)

with open(filename) as fopen:
    d_ = json.load(fopen)

In [24]:
segments, temp = [], [d_['word_timestamps'][0]]
last_t = d_['word_timestamps'][0]['end']
for c in d_['word_timestamps'][1:]:
    if (c['start'] - last_t) > 0.2:
        segments.append(temp)
        temp = []
        
    last_t = c['end']
    temp.append(c)
    
if len(temp):
    segments.append(temp)
    
ts = []
for s in segments:
    start = min(timestamps, key=lambda t: abs(t - s[0]['start']))
    end = min(timestamps, key=lambda t: abs(t - s[-1]['end']))
    w = ' '.join([c['text'] for c in s])
    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
    ts.append(t)
            


['<|0.06|> Bagaimanakah kita boleh mengendalikan ralat<|2.46|>',
 '<|2.84|> pada laman web menggunakan JavaScript?<|4.88|>']

In [23]:
s[0]['start'], s[-1]['end']

(2.84, 4.88)

In [38]:
def loop(rows):
    rows, _ = rows
    result = []
    for r in tqdm(rows):
        f = r['audio_filename']
        new_f = f.replace('/', '_').replace('.mp3', '.json').replace('.wav', '.json')
        filename = os.path.join('force_alignment', new_f)
        if not os.path.exists(filename):
            continue
        
        with open(filename) as fopen:
            d_ = json.load(fopen)
            
        segments, temp = [], [d_['word_timestamps'][0]]
        last_t = d_['word_timestamps'][0]['end']
        for c in d_['word_timestamps'][1:]:
            if (c['start'] - last_t) > 0.25:
                segments.append(temp)
                temp = []

            last_t = c['end']
            temp.append(c)

        if len(temp):
            segments.append(temp)

        ts = []
        for s in segments:
            start = min(timestamps, key=lambda t: abs(t - s[0]['start']))
            end = min(timestamps, key=lambda t: abs(t - s[-1]['end']))
            w = ' '.join([c['text'] for c in s])
            t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
            ts.append(t)

        ts = ''.join(ts)
        new_text = text = f"<|startoftranscript|><|ms|><|transcribe|>{ts}<|endoftext|>"
        
        result.append({
            'new_text': new_text,
            'audio_filename': f.split('speech-instructions/')[1]
        })
    return result

In [39]:
r = loop((d[:100], 0))

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3048.01it/s]


In [40]:
r[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.06|> Bagaimanakah kita boleh mengendalikan ralat<|2.46|><|2.84|> pada laman web menggunakan JavaScript?<|4.88|><|endoftext|>',
 'audio_filename': 'short-coding-2/0.mp3'}

In [41]:
r = mp.multiprocessing(d, loop, cores = 10)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [42]:
len(r)

510940

In [48]:
r[-1000]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|1.08|> Mengapa terdapat<|2.28|><|2.66|> tuduhan kronisme<|3.70|><|4.82|> dan nepotisme<|5.64|><|6.14|> terhadap kerajaan Malaysia,<|7.48|><|8.20|> terutamanya<|8.90|><|9.48|> berkaitan dengan<|10.32|><|10.62|> pelantikan politik peringkat tinggi?<|13.08|><|endoftext|>',
 'audio_filename': 'partition-instructions-part-12/28940.mp3'}

In [44]:
import pandas as pd

pd.DataFrame(r).to_parquet('gather-force-alignment-segment-timestamps.parquet')

In [45]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="gather-force-alignment-segment-timestamps.parquet",
    path_in_repo="data/malaysian_speech_instructions_segment_timestamp-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

gather-force-alignment-segment-timestamps.parquet:   0%|          | 0.00/40.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/040aecb0ef60bea32a1c43655127b38e92098afd', commit_message='Upload data/malaysian_speech_instructions_segment_timestamp-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='040aecb0ef60bea32a1c43655127b38e92098afd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)